<a href="https://colab.research.google.com/github/somilasthana/deeplearningtutorial/blob/master/NLP_Tensorflow_IMDB_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
tf.__version__

import numpy as np
!pip install tensorflow==2.0.0-beta0 # if you dont do it then imdb data iteration fails

tf.__version__
#tf.enable_eager_execution() # Enable if not using 2.0.0-beta0

'2.0.0-beta0'

In [0]:
import tensorflow_datasets as tfds

imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

In [0]:
train_data, test_data = imdb["train"], imdb["test"]

In [4]:
train_data, test_data

(<_OptionsDataset shapes: ((), ()), types: (tf.string, tf.int64)>,
 <_OptionsDataset shapes: ((), ()), types: (tf.string, tf.int64)>)

In [0]:
training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

In [0]:
for s, l in train_data:
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())
  
for s,l in test_data:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())

In [9]:
print(training_sentences[10])

b'I haven\'t seen this film in years so my knowledge is a little rusty. I do remember thinking that this film is twice the film of Braveheart. It is simply more realistic and has more believable characters. Ridb Roy looks like one would imagine Rob Roy to look like, messy hair and beard with simple clothing. Also the Liam Nesson has a Celtic look about him, he looks like a Scot and more importantly looks like Rob Roy. It\'s a comparison which angers some people but compare him to Mel Gibson as William Wallace. Gibson is supposedly playing a man who\'s legend has caused him to be described as a 6 foot 7 giant while Gibson is almost a foot shorter. The story contains a little romance, conspiracy and an underdog story. Sound anything like Braveheart? But instead of a film that cries out "freedom", liberty and nationalism we get a film which says honour, love and justice. This makes it a more interesting film. Much like Gladiator. The cast is fantastic and Liam Nesson is a very strong lead

In [10]:
print(training_labels[10])

1


In [0]:
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [0]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok="<OOV>"

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [0]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)

In [0]:
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length)

In [19]:
reverse_word_index = { value: key for (key, value) in word_index.items() }

def decode_review(text):
  return ' '.join([reverse_word_index.get(i, '?') for i in text ])

print(decode_review(padded[1]))
print(training_sentences[1])

b'i have no idea what the other reviewer is talking about this was a wonderful movie and created a sense of the era that feels like time travel the characters are truly young mary is a strong match for byron claire is juvenile and a tad annoying <OOV> is a convincing beaten down <OOV> all are beautiful curious and <OOV> not the frightening <OOV> they are in gothic br br gothic works as an independent piece of shock film and i loved it for different reasons but this works like a <OOV> and <OOV> film and was from my <OOV> the best capture of what the summer must have felt like romantic yes but completely <OOV> my interest in the
b'I have no idea what the other reviewer is talking about- this was a wonderful movie, and created a sense of the era that feels like time travel. The characters are truly young, Mary is a strong match for Byron, Claire is juvenile and a tad annoying, Polidori is a convincing beaten-down sycophant... all are beautiful, curious, and decadent... not the frightening

In [0]:
# Building Model

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [0]:
model.compile(
    loss='binary_crossentropy',
    optimizer = 'adam',
    metrics=['accuracy']
)

In [48]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
global_average_pooling1d_2 ( (None, 16)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 24)                408       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 25        
Total params: 160,433
Trainable params: 160,433
Non-trainable params: 0
_________________________________________________________________


In [49]:
num_epochs =10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 4s 164us/sample - loss: 0.5039 - accuracy: 0.7709 - val_loss: 0.3503 - val_accuracy: 0.8519
Epoch 2/10
25000/25000 [==============================] - 4s 149us/sample - loss: 0.3032 - accuracy: 0.8721 - val_loss: 0.3363 - val_accuracy: 0.8529
Epoch 3/10
25000/25000 [==============================] - 4s 150us/sample - loss: 0.2486 - accuracy: 0.9011 - val_loss: 0.3493 - val_accuracy: 0.8502
Epoch 4/10
25000/25000 [==============================] - 4s 150us/sample - loss: 0.2151 - accuracy: 0.9173 - val_loss: 0.3766 - val_accuracy: 0.8432
Epoch 5/10
25000/25000 [==============================] - 4s 150us/sample - loss: 0.1887 - accuracy: 0.9310 - val_loss: 0.4060 - val_accuracy: 0.8393
Epoch 6/10
25000/25000 [==============================] - 4s 150us/sample - loss: 0.1701 - accuracy: 0.9387 - val_loss: 0.4472 - val_accuracy: 0.8294
Epoch 7/10
25000/25000 [==========================

In [32]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences(sentence)
print(sequence)

[[11], [], [1431], [968], [4], [1537], [1537], [4737], [], [790], [2015], [11], [2923], [2190], [], [790], [2015], [11], [579], [], [11], [579], [], [4], [1783], [4], [4508], [11], [2923], [1277], [], [], [2015], [1005], [2923], [968], [579], [790], []]


In [0]:
e = model.layers[0]

weights = e.get_weights()[0]

In [40]:
weights.shape

(10000, 16)

In [0]:
import io

for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  print(word + "\n")
  print('\t'.join([str(x) for x in embeddings]) + "\n")
  
  
  """
  <OOV>

0.049393594	0.12212473	-0.14989318	-0.22841957	0.11194647	0.074225776	0.33750403	-0.3389223	0.25071773	-0.13969828	-0.39301983	0.29610136	0.28573972	-0.14618364	-0.17800578	0.25514895

the

-0.012467001	0.1363901	-0.083791964	-0.28604594	0.04918319	0.053041406	0.32991502	-0.27210024	0.30938575	-0.07874642	-0.28730544	0.36824962	0.37688512	-0.19305907	-0.17293388	0.20891072

an
  """
  